# A slightly deeper dive
Now we'll try to cover some "intermediate" features of the language.


# 1. Python magic methods

Before we look at magic methods, here is a quick overview of the different types of method naming conventions :

```
1. _method    : To prevent automatic import due to a "from xyz import *" statement.
2. __method   : To mark as a private method.
3. method_    : To deal with reserved words
4. __method__ : magic functions, hooks that are triggered on various builtin operators and functions.
```

In [2]:
class A(object):
    def __foo(self):
        print "A foo"
    def class_(self):
        self.__foo()
        print self.__foo.__name__
    def __doo__(self):
        print "doo"
        
a = A()
print hasattr(a, '__foo') # where has this gone?
a.class_()
a.__doo__()

False
A foo
__foo
doo


So we can see here that we cannot access the private method outside the class, this is due to name mangling. The members can be inspected using the builtin `dir` method.

In [7]:
print dir(a)

['_A__foo', '__class__', '__delattr__', '__dict__', '__doc__', '__doo__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'class_']


As we see, the name has changed to `_A_foo` and that is the only reason it is "private" if we explicitly call it by it's mangaled name, it is very much accessible.

In [6]:
a._A__foo()

A foo


It's worth noting that the `__magic_method__` format does not do anything special unless we use the predefined names. It is also strongly advised that you only override the builtin magic methods and not re define your own as I had just done previously!

In [10]:
class P(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        return P(self.x + other.x, self.y + other.y)
    
    def __gt__(self, other):
        #if both x and y component is greater than the other object's x and y
        return (self.x > other.x) and (self.y > other.y)
    
    def __str__(self):
        return "x : %s, y : %s" % (self.x, self.y)
    
p1 = P(0,0)
p2 = P(3,4)
p3 = P(1,3)

print p3 + p2
print p1 > p2
print p2 > p1


x : 4, y : 7
False
True


You can even add stuff like **slicing capabilities**

In [18]:
class Seq(object):
    def __getitem__(self, i):
        if type(i) is slice:
            # this has edge case issues, but just a demo!
            return list(range(i.start, i.stop))
        else:
            return i

s = Seq()
print s[5]
print s[-4]
print s[2:5]

5
-4
[2, 3, 4]


**Note** We have covered a very small subset of all the "magic" functions! Please do have a look at the official python docs for the exhaustive reference!

# 2. Comprehension

Python comprehensions gives us interesting ways to populate builtin data structures, *in terms of expressions*, as mathematicians would do. Comprehensions are a paradigm borrowed from functional languages, and provides a great deal of syntactic sugar. 

In [30]:
l = [i for i in range(0,5)]
l2 = [i*i for i in range(0,5)]
print l
print l2

[0, 1, 2, 3, 4]
[0, 1, 4, 9, 16]


We can also define slightly more complex expressions with the use of if statements and nested loops.

In [22]:
l = [i for i in range(0,5) if i % 2 ==0]
print l

[0, 2, 4]


In [24]:
# get all combinations where x > y and x, y < 5
xy = [ (x, y) for x in range (0,5) for y in range (0, 5) if x > y]
print xy

[(1, 0), (2, 0), (2, 1), (3, 0), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2), (4, 3)]


In [25]:
# we can even call functions
l = [x.upper() for x in "hello"]
print l


['H', 'E', 'L', 'L', 'O']


In [29]:
# creating lists of lists is also a synch
gre = "hello how are you doing?"
[[s.lower(), s.upper(), len(s)] for s in gre.split()]

[['hello', 'HELLO', 5],
 ['how', 'HOW', 3],
 ['are', 'ARE', 3],
 ['you', 'YOU', 3],
 ['doing?', 'DOING?', 6]]

In [39]:
# nested comprehensions - we can do it, but it may not be very readable
matrix = [[i+x for i in range(3)] for x in range(3)]
print matrix

[[0, 1, 2], [1, 2, 3], [2, 3, 4]]


In [34]:
# we can also have a comprehension for dicts
d = {x : x**2 for x in range(5)}
print d

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}


# 3. Functional parts

There are a lot of concepts borrowed from functional languages to make the code look more elegant. List comprehension was just scratching the surface. We have builtin helpers such as `lambda, filter, zip, map, all, any` to help us write cleaner code! Other than the builtin components we have functools (which I won't be covering) which even helps us with partial functions and currying!

In [58]:
# lambda is used when you need anonymous functions defined as an expression
# in this example you could define a function and pass it to foo, or use the lambda
# in this case the lambda is neater.
# lambdas can take in n number of params, and the body is a single expression that is also the return value

def foo(list_, func):
    l = []
    for i in list_:
        l.append(func(i))
    return l

def sq(i):
    return i**2

l = [i for i in range(5)]
print foo(l, sq)
print foo(l, lambda x : x**2)


[0, 1, 4, 9, 16]
[0, 1, 4, 9, 16]


In [59]:
class P(object):
    def __init__(self, x):
        self.x = x
    def __str__(self):
        return "x : %s" % self.x

l = [P(5), P(2), P(1), P(4), P(3)]
l.sort(cmp=lambda x, y: x.x - y.x)
for p in l : print p # [str(p) for p in l]
    
# there are many more complex and cryptic ways to use (exploit) lambdas, 
#     you can search for it online if you are interested
# check lambda with multiple args
# lambda *x : sys.stdout.write(" ".join(map(str, x)))


x : 1
x : 2
x : 3
x : 4
x : 5


In [62]:
# filter is a function that takes an interable and a callable 
#  applies the function to each element,i.e. ret = func(element)
#  and returns a list with elements for which 'ret' was true

l = range(0,10)
l = filter(lambda x : x%2==0, l)
print l, type(l)

[0, 2, 4, 6, 8] <type 'list'>


In [65]:
# zip is to sew together a bunch of iterables
# the list generated is of the minimum size of all the iterators that have gone in!

a = [1,2,3,4,5]
b = (0,4,6,7)
c = {1:'a', 7:'b', 'm':'v'}

print zip(a,b,c)


[(1, 0, 1), (2, 4, 'm'), (3, 6, 7)]


Though zip looks trivial it is a fairly important operation for mathematical algorithms - matrices, curve fitting, interpolation, pattern recognition, that sort of thing. Also very important in engineering applications like digital signal processing where much of what you do is combine multiple signals or apply linear transforms to them - both are based on the sample index, hence, zip it.

It would be a pain to reimplement it everytime taking care of all edge cases, etc!

In [67]:
# map - takes in a iterable and callable - applies the callable to each element of the iterable
#  returns a new list with each element being the return value of "callable(elem)"

print map(lambda x: x**2, range(10))



[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [69]:
# map is extremely useful as a shorthand for "applying" a function across an iterable,
#  especially in conjunction with lambda
import sys
my_print = lambda *x : sys.stdout.write(" ".join(map(str, x)))
my_print("hello", "how are you", 1234)

hello how are you 1234

# 4. Iterables

Lists, dicts, tuples are iterables. i.e. we can "iterate" through them! Any object that supports the iterator protocol is a iterator. The iterator protocol states that the object should override the `__iter__` magic method that returns an object that has a `.next()` method and raises a `StopIteration exception`. 

There are 4 key ways to create an iterable - 

```
1. Iterators - classes that override __iter__ and next()
2. Generator functions - functions that yield
3. Generator expressions
4. overriding the __getitem__ magic method.
```


In [75]:
# 1. Iterators

class myitr(object):
    def __init__(self, ulimit=5):
        self.limit = ulimit
    def __iter__(self):
        self.index = 0
        return self
    def next(self):
        if self.index < self.limit:
            self.index += 1
            return self.index
        else:
            raise StopIteration
            
itr = myitr()
for i in itr:
    print i

1
2
3
4
5


In [76]:
# 2. Generators

def gen(lim):
    i = 0
    while i < lim:
        yield i
        i = i + 1
        
for i in gen(5):
    print i

0
1
2
3
4


In [78]:
# 3. Generator expression

def seq(num):
    return (i**2 for i in range(num))

for i in seq(5):
    print i


0
1
4
9
16


In [80]:
# 4. Overriding __getitem__

class Itr(object):
    def __init__(self, x):
        self.x = x
        
    def __getitem__(self, index):
        if index < self.x:
            return index
        else:
            raise StopIteration


for i in Itr(5):
    print i

0
1
2
3
4


# 5. Decorators

Before we start with decorators, we need to know a bit about closures. A closure is a function object that remembers values in enclosing scopes regardless of whether those scopes are still present in memory. Most common case is when we define a function within a function, and return the inner function. If the inner function definition uses variables / values in the outer function, it maintains the references to those even after it is returned (and no longer in the scope of the outer function).

In [84]:
# closure example - raised_to_power returns a fn that takes a variable and raises to the power 'n'
#  'n' is passed only once - while defining the function!

def raised_to_power(n):
    def fn(x):
        return x**n
    return fn
    
p2 = raised_to_power(2)
p3 = raised_to_power(3)

print p2(2), p2(3) # still remembers that n=2
print p3(2), p3(3) # still remembers that n=3

4 9
8 27


In [96]:
# have to be cautious!

def power_list(n):
    '''returns list of fn, each raises to power i, where i : 0 --> n'''
    fn_list = []

    def fn(x):
        return x**i
    
    for i in range(n):
        # doesn't matter if fn was defined here either
        fn_list.append(fn)

    return fn_list

for j in power_list(4):
    print j(2) # prints 2 power 3, 4 times
    

8
8
8
8


In [82]:
# decorator is just a nicer way of defining a closure - more syntactic sugar

def deco(fn):
    def new_fn(*args, **kwargs):
        print "entring function", fn.__name__
        ret = fn(*args, **kwargs)
        print "exiting function", fn.__name__
    return new_fn

@deco
def foo(x):
    print "x : ", x
    
foo(4)



entring function foo
x :  4
exiting function foo


In [99]:
# Another example

def add_h1(fn):
    def nf(pram):
        return "<h1> " + fn(pram) + " </h1>"
    return nf

@add_h1
def greet(name):
    return "Hello {0}!".format(name)

print greet("Nutanix")

<h1> Hello Nutanix! </h1>


In [104]:
# decorator that takes parameter

def add_h(num):
    def deco(fn):
        # this is the decorator for a specific 'h'
        def nf(pram):
            return "<h%s> "%num + fn(pram) + " </h%s>"%num
        return nf
    return deco

@add_h(3)
def greet(name):
    return "Hello {0}!".format(name)
print greet("Nutanix")


# we can have multiple decorators as well
@add_h(2)
@add_h(4)
def greet2(name):
    return "Hello {0}!".format(name)

print greet2("Nutanix")
        

<h3> Hello Nutanix! </h3>
<h2> <h4> Hello Nutanix! </h4> </h2>


# 6. More OO

Let's take another look at classes and OO in python. We'll start with multiple inheritance (or mixins).


### Mixins

Let's start with this inheritance model -
```
    A
   / \
  B   C
   \ /
    D
```

In [106]:
class A(object):
    def __init__(self):
        print "A.init"        
    def foo(self):
        print "A.foo"
        
class B(A):
    def __init__(self):
        print "B.init"
    def foo(self):
        print "B.foo"

class C(A):
    def __init__(self):
        print "C.init"
    def foo(self):
        print "C.foo"
        
class D(B, C):
    def __init__(self):
        print "D.init"
    #def foo(self):
    #    print "D.foo"
    
class E(C, B):
    def __init__(self):
        print "E.init"

d = D()
d.foo() 

e = E()
e.foo()

# we see that fn lookup's happen in the order of declaration of parent in the child's definition.

D.init
B.foo
E.init
C.foo


What if the mixin is slightly more complex? (Note, how ever complex stuff gets - which it shouldn't, python will never let you create a circular dependency!

```
       A
     /  \
     B  C
     | /|
     D/ |
     |  |
      \ |
       E
       
```

In [111]:
class A(object):
    def __init__(self):
        print "A.init"        
    def foo(self):
        print "A.foo"
        
class B(A):
    def __init__(self):
        print "B.init"
    def foo(self):
        print "B.foo"

class C(A):
    def __init__(self):
        print "C.init"
    def foo(self):
        print "C.foo"
        
class D(C):
    def __init__(self):
        print "D.init"
    def foo(self):
        print "D.foo"
    
class E(D, C): # you can't have (C, D) - TypeError: Cannot create a consistent MRO
    def __init__(self):
        print "E.init"

e = E()
e.foo()
E.__mro__

# so what's mro - (explain in live session)

E.init
D.foo


(__main__.E, __main__.D, __main__.C, __main__.A, object)

**NOTE** MRO is also the reason why `super()` is called in the manner it is. You need both the class and the object to traverse the next parent in the MRO!

### Attribute access hooks
Next let's have a look at two magic functions which deal with object variable access, `__getattr__` and `__setattr__`.

In [127]:
class A(object):
    def __init__(self, x):
        self.x = x
    def __getattr__(self, val):
        print "getattr val :", val, type(val)
        return val

a = A(3)
print "X :", a.x # getattr not called for x
ret = a.y
print "Y :", ret


X : 3
getattr val : y <type 'str'>
Y : y


In [129]:
class A(object):
    def __init__(self, x):
        self.x = x
    def __getattr__(self, val):
        print "getattr"
        return val
    def __setattr__(self, name, val):
        print "setattr"
        if name == 'x':
            self.__dict__[name] = val

a = A(3)
print a.x
print a.y

# setattr is called for both
a.y = 5
a.x = 5

setattr
3
getattr
y
setattr
setattr


### Callable objects

You can make an object callable (a functor) by overriding the magic `__call__` method. You can call the object like a function and the `__call__` method will be called instead. This is useful when you want to have more complex functionality (like state) plus data, but want to keep the syntactic sugar / simplicity of a function!

In [132]:
class MulBy(object):
    def __init__(self, x):
        self.x = x
    def __call__(self, n):
        print "here!"
        return self.x * n
    
m = MulBy(5)
print m(3)

here!
15


### \__new\__ vs \__init\__

Until now we never bothered to see how / when the python objects were created. The `__init__` function just deals with handling the initialization of the object, the actual creation happens within `__new__`, which can be overridden.

From the python mailing list

```
Use __new__ when you need to control the creation of a new instance.
Use __init__ when you need to control initialization of a new instance.

__new__ is the first step of instance creation.  It's called first,
and is responsible for returning a new instance of your class.  In
contrast, __init__ doesn't return anything; it's only responsible for
initializing the instance after it's been created.
```


In [137]:
class X(object):
    def __new__(cls, *args, **kwargs):
        print "new"
        print args, kwargs
        return object.__new__(cls)
        
    def __init__(self, *args, **kwargs):
        print "init"
        print args, kwargs
        
x = X(1,2,3,a=4)

new
(1, 2, 3) {'a': 4}
init
(1, 2, 3) {'a': 4}


**This approach is rather useful for the factory design pattern!**

In [150]:
class WindowsVM(object):
    def __init__(self, state="off"):
        print "New windows vm. state : %s" %state
    def operation(self):
        print "windows ops"
        
class LinuxVM(object):
    def __init__(self, state="off"):
        print "New linux vm. state : %s" %state
    def operation(self):
        print "linux ops"

class VM(object):
    MAP = {"Linux" : LinuxVM, "Windows": WindowsVM}
    
    def __new__(self, vm_type, state="off"):
        # return object.__new__(VM.MAP[vm_type]) #--doesn't call init of other class
        vm = object.__new__(VM.MAP[vm_type])
        vm.__init__(state)
        return vm


vm1 = VM("Linux")
print type(vm1)
vm1.operation()
print ""
vm2 = VM("Windows", state="on")
print type(vm2)
vm2.operation()

New linux vm. state : off
<class '__main__.LinuxVM'>
linux ops

New windows vm. state : on
<class '__main__.WindowsVM'>
windows ops


# 7. Properties

Properties are ways of adding behaviour to instance variable access, i.e. trigger a function when a variable is being accessed. This is most commonly used for getters and setters.

In [159]:
# simple example 
class C(object):
    def __init__(self):
        self._x = None

    def getx(self):
        print "getx"
        return self._x
    
    def setx(self, value):
        print "setx"
        self._x = value
        
    def delx(self):
        print "delx"
        del self._x
        
    x = property(getx, setx, delx, "I'm the 'x' property.")
    
c = C()
c.x = 5 # so when we use 'x' variable of a C object, the getters and setters are being called!
print c.x
del c.x

print C.x

setx
getx
5
delx


In [168]:
# the same properties can be used in form of decorators!
class M(object):
    def __init__(self):
        self._x = None

    @property
    def x(self):
        print "getx"
        return self._x
    
    @x.setter
    def x(self, value):
        print "setx"
        self._x = value
    
    @x.deleter
    def x(self):
        print "delx"
        del self._x

m = C()
m.x = 5 
print m.x
del m.x


5


So how does this magic happen? how do properties work? It so happens that properties are data discriptors! Discriptors are objects that have a `__get__`, `__set__`, `__del__` method. When accessed as a member variable, the corresponding function gets called. Property is a class that implements this discriptor interface, there is nothing more to it!

In [169]:
# This is a pure python implementation of property

class Property(object):
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
    
# during the live session, explain how this maps to the previous decorator version of property.

# 8. Metaclasses

Meta classes are classes that create new classes (or rather a class whose instances are classes themselves!). They are useful when you want to dynamically create your own types. Eg. when you have to create classes based on a description file (XML) - like in the case of some libraries built over WSDL (PyVmomi). Or in the case when you want to dynamically mix two or more types of classes to create a new one. (eg. a VM type, a OS type and an interface type - used in NuTest framework developed by the automation team). 

Another reason is to enforce some checking / have restrictions in the user defined classes. Like in the case of Abstract Base Class (ABCMeta). The meta class which creates the user defined class can run some pre checks (whether certain function are defined, etc) and some pre processing (adding new methods, etc) if required!

In [175]:
class MyMet(type):
    """Here we see that MyMet doesn't inherit 'object' but rather 'type' class - the builtin metaclass
    """
    def __new__(cls, name, bases, attrs):
        """
        Args:
          name (str) : name of the new class being created
          bases (tuple) : tuple of the classes which are the parents of cls
          attrs (dict) : the attributes that belong to the class
        """
        print "In new"
        print name
        print bases
        print attrs
        return super(MyMet, cls).__new__(cls, name, bases, attrs)
        
    def __init__(self, *args, **kwargs):
        print "In init"
        print self
        print args
        print kwargs
        
class Me(object):
    __metaclass__ = MyMet
    
    def foo(self):
        print "I'm foo"
    

m = Me()
m.foo()

In new
Me
(<type 'object'>,)
{'__module__': '__main__', 'foo': <function foo at 0x10fd0f5f0>, '__metaclass__': <class '__main__.MyMet'>}
In init
<class '__main__.Me'>
('Me', (<type 'object'>,), {'__module__': '__main__', 'foo': <function foo at 0x10fd0f5f0>, '__metaclass__': <class '__main__.MyMet'>})
{}
I'm foo


In this case we see that 'm' which is an instance of 'Me' works as expected. Here we're using the metaclass to just print out the flow, but we can do much more!

Also if you note we see that the args to `__init__` are the same as args to `__new__`, which is again as expected.

In [193]:
class MyMet(type):
    """Here we see that MyMet doesn't inherit 'object' but rather 'type' class - the builtin metaclass
    """
    def __new__(cls, name, bases, attrs):
        """
        Args:
          name (str) : name of the new class being created
          bases (tuple) : tuple of the classes which are the parents of cls
          attrs (dict) : the attributes that belong to the class
        """
        print "In new"
        print name
        print bases
        print attrs
        def foo(self):
            print "I'm foo"
        attrs['foo'] = foo
        return super(MyMet, cls).__new__(cls, name, bases, attrs)
        
    def __init__(self, name, bases, attrs):
        print "In init"
        print self # actually the object being created
        print name
        print bases
        print attrs
        def bar(self):
            print "I'm bar"
        setattr(self, "bar", bar)
        
    def test(self):
        print "in test"
        
    #def __call__(self):
    #    print "self :", self
    # Note : If I override call here, then I have to explicitly call self.__new__
    #        otherwise it is completely skipped. Normally a class calls type's __call__
    #        which re-routes it to __new__ of the class
         
        
class Me(object):
    __metaclass__ = MyMet




print "\n-------------------------------\n"

m = Me()
print type(Me) # not of type 'type' anymore!
m.foo()
m.bar()
# print m.test --attribute error
Me.test()

In new
Me
(<type 'object'>,)
{'__module__': '__main__', '__metaclass__': <class '__main__.MyMet'>}
In init
<class '__main__.Me'>
Me
(<type 'object'>,)
{'__module__': '__main__', 'foo': <function foo at 0x10fd0ff50>, '__metaclass__': <class '__main__.MyMet'>}

-------------------------------

<class '__main__.MyMet'>
I'm foo
I'm bar
in test


**Note**

What the `__metaclass__` does is it tells the interpreter to parse the class in question, get the name, the attribute dictionary and the base classes and create it using a 'type' type, in this case, the MyMet class. In it's most primitive form that's how classes are created, using the 'type' inbuilt class. We use this a lot to dynamically mix classes in NuTest!

In [198]:
class A(object):
    def __init__(self):
        print "init A"
    def foo(self):
        print "foo A"
    def bar(self):
        print "bar A"
        
class B(object):
    def __init__(self):
        print "init B"
    def doo(self):
        print "doo B"
    def bar(self):
        print "bar B"
        
def test(self):
    print "Self : ", self

Cls = type("C", (A,B), {"test": test})

c = Cls()
print Cls
print Cls.__name__, type(Cls)
print c

init A
<class '__main__.C'>
C <type 'type'>


In [196]:
c.foo()
c.bar()
c.doo()
c.test()

foo A
bar A
doo B
Self :  <__main__.C object at 0x10fcff750>
